# data_to_analysis_unmatched.ipynb


- From Data to Unconditional Analysis

We start again from the datasets, and we go to margins. This dataset is merged with a dataset of politicians with the margins (conditional on the margin being $<x$ %, but not necessarily from the same elections. 

From this, we can already make an Y|Margin plot. 

In [1]:
# Load the libraries

import pandas as pd
import numpy as np
import re
from pandas_ods_reader import read_ods

from scipy import stats
from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.compat import lzip
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import pyplot as plt

from tqdm import tqdm

import fuzzywuzzy as fw
from fuzzy_match import match
from fuzzy_match import algorithims
from fuzzywuzzy import process

/home/bas/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## From Data to unmatched Analysis

First step: 

- Import the data

- The data comes straight away from the google drive sheets, so this code can be rerun every time new observations are obtained. 

- In contrast to the matched analysis, we don't filter on elections for nonpoliticians that were only found, because politicians that won in close elections for which nonpoliticians weren't found should still be in the sample. 


In [2]:
data1 = pd.read_csv("../Administration/new_data_entry_file.csv").iloc[:,1:]
data2 = pd.read_csv("../Administration/new_data_entry_file_lessclose.csv")

nonpoliticians = pd.concat([data1, data2])

nonpoliticians['jaar'] = nonpoliticians['Verkiezingdatum'].astype(str).str.split('/').apply(lambda x : x[2])
nonpoliticians.head(2)

,Naam,District,Verkiezingdatum,InTKEK,Gevonden,Gebdatum,Sterfdatum,Sterfplaats,Vermogen,Link1,Link2,Polpartij,Opm,Unnamed: 13,jaar
0,mr. J. van Riemsdijk,Almelo,02/01/1849,nee,ja,NaN,01/11/1879,Almelo,29618.27,https://www.genealogieonline.nl/en/genealogie-...,NaN,NaN,NaN,NaN,1849
1,mr. H. van Sonsbeeck,Enschede,30/11/1848,nee,nee,NaN,29/11/1865,Heino,NaN,NaN,NaN,NaN,Voor 1877,NaN,1848


## Second step

We now import a list of all elections and all politicians, and we identify the margin of each candidates based on the number of zetels in each election (column 'Aantal zetels'). 

In [3]:
def decode_accents(name):
    try:
        return name.encode('latin-1').decode('raw_unicode_escape').encode('latin-1').decode('utf-8')
    except:
        return name

In [4]:
all_candidates_elections = pd.read_csv("../Data/elections/election_results_details.csv", encoding='utf-8-sig')
all_candidates_elections["Naam"] = all_candidates_elections["Naam"].apply(lambda x : decode_accents(x))
all_candidates_elections['Aantal stemmen'] = pd.to_numeric(all_candidates_elections['Aantal stemmen'], errors="coerce")

all_candidates_elections.head(2)

,Unnamed: 0,Naam,Aanbevolen door,Aantal stemmen,Procentueel,District,Verkiezingdatum,Type,Omvang electoraat,Opkomst,Aantal stembriefjes,Aantal stemmen geldig,Aantal stemmen blanco,Aantal zetels,Kiesdrempel
0,0,mr. B. Wichers,NaN,700.0,83.73%,Groningen,30/11/1848,algemeen,1191,838,838,836,0,1,418
1,1,W.L. de Sturler,NaN,34.0,4.07%,Groningen,30/11/1848,algemeen,1191,838,838,836,0,1,418


## Recompute the opkomst

Because it is not correct in a couple of districts. 

In [5]:
aantalstemmen = all_candidates_elections.groupby(['District', 'Verkiezingdatum']).agg({'Aantal stemmen':'sum'}).reset_index()

#aantalstemmen
all_candidates_elections = all_candidates_elections.merge(aantalstemmen, how='left', 
                               left_on = ['District', 'Verkiezingdatum'],
                              right_on = ['District', 'Verkiezingdatum'])

del(all_candidates_elections['Opkomst'])

all_candidates_elections = all_candidates_elections.rename(columns={'Aantal stemmen_y':'Opkomst',
                                        'Aantal stemmen_x':'Aantal stemmen'})

In [6]:
def to_int(x):
    try:
        return int(x)
    except:
        return None
    
#all_candidates_elections['Aantal stemmen'] = all_candidates_elections['Aantal stemmen'].apply(to_int)
all_candidates_elections['Aantal zetels'] = all_candidates_elections['Aantal zetels'].apply(to_int)
all_candidates_elections['Aantal stemmen geldig'] = all_candidates_elections['Aantal stemmen geldig'].apply(to_int)

## Attention

In this function, pay attention how I define margin. Surprisingly, in almost no studies that use close elections, much attention is paid to this detail, presumably because it is all american and these people do not care much about elections with more than two candidates, so they always put the margin at 50%. 

My margin is defined as:

$$
\text{margin} = \begin{cases}
\text{Votes}_i - \text{# Votes Next Loser} &\mbox{ if } i \text{ is a winner} \\
\text{Votes}_i - \text{# Votes Next Winner} &\mbox{ if } i \text{ is a loser}
\end{cases}
$$


In [7]:
# Now in this function, we reproduce the dataset and create the margin:

#df is supposed to be all_candidates_elections

def get_margin(df):
    ## First, create a dataframe that contains only the number of seats per election
    zetels = all_candidates_elections.groupby(['District', 'Verkiezingdatum']).agg({'Aantal zetels': 'mean'})

    margins = []

    for i in tqdm(range(len(df))):
    
        try:
            distr = all_candidates_elections.loc[i,'District']
            date = all_candidates_elections.loc[i,'Verkiezingdatum']
            number_of_seats = zetels.loc[distr, date][0]
            
        # Filter the dataframe according to district and election date - and select the observation that is the marginal winner
    
            amountvotes_marginalwinner = (all_candidates_elections[(all_candidates_elections['District'] == distr) & (all_candidates_elections['Verkiezingdatum'] == date)].
             sort_values('Aantal stemmen', ascending = False).reset_index(drop=True).loc[number_of_seats-1,'Aantal stemmen'])
        
            amountvotes_marginalloser = (all_candidates_elections[(all_candidates_elections['District'] == distr) & (all_candidates_elections['Verkiezingdatum'] == date)].
             sort_values('Aantal stemmen', ascending = False).reset_index(drop=True).loc[number_of_seats,'Aantal stemmen'])
            
            # If I am the winner, I get a positive margin
            if all_candidates_elections.loc[i,'Aantal stemmen'] >= amountvotes_marginalwinner:
                margin = (all_candidates_elections.loc[i, 'Aantal stemmen'] - amountvotes_marginalloser) /all_candidates_elections.loc[i,'Opkomst']
            
            # If I am the loser, I get a negative margin
            else: 
                margin = (all_candidates_elections.loc[i,'Aantal stemmen'] - amountvotes_marginalwinner)/all_candidates_elections.loc[i,'Opkomst']
        
            margins.append(margin)
        
        except:
            
            margins.append(999)

    
    df['margin'] = pd.Series(margins)
    
    return(df)

In [8]:
df_margins = get_margin(all_candidates_elections)

100%|██████████| 8238/8238 [00:42<00:00, 193.78it/s]


In [9]:
# Only very few people for whom we were not able to compute the margins because of missing data
df_margins[df_margins['margin'] == 999]

,Unnamed: 0,Naam,Aanbevolen door,Aantal stemmen,Procentueel,District,Verkiezingdatum,Type,Omvang electoraat,Aantal stembriefjes,Aantal stemmen geldig,Aantal stemmen blanco,Aantal zetels,Kiesdrempel,Opkomst,margin
141,141,mr. J.K. baron van Goltstein,NaN,532.0,96.20%,Utrecht,30/11/1848,algemeen,651,554,553.0,1,1.0,277,532.0,999.0
257,257,mr. J.A. Mutsaers,NaN,607.0,93.53%,Tilburg,30/11/1848,algemeen,795,650,649.0,1,1.0,325,607.0,999.0
339,339,dr. R. Westerhoff,NaN,743.0,92.30%,Appingedam,12/06/1860,periodiek,3138,823,805.0,4,1.0,403,743.0,999.0
397,397,mr. H. Goeman Borgesius,(AH)*,618.0,87.17%,Winschoten,14/06/1881,periodiek,2749,723,709.0,12,1.0,355,618.0,999.0
404,404,mr. H.L. Drucker,NaN,606.0,-,Winschoten,14/09/1887,herstemming,2923,-,1110.0,-,0.0,-,1110.0,999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8215,8215,J.H.J. Beckers,Ka,NaN,enige kandidaat,Sittard,15/06/1917,algemeen/enkelvoudig,10264,-,NaN,-,NaN,-,0.0,999.0
8216,8216,dr. W.H. Nolens,Ka,NaN,enige kandidaat,Venlo,15/06/1917,algemeen/enkelvoudig,9190,-,NaN,-,NaN,-,0.0,999.0
8217,8217,mr.dr. H.A.G. van Groenendael,Ka,NaN,enige kandidaat,Weert,15/06/1917,algemeen/enkelvoudig,8626,-,NaN,-,NaN,-,0.0,999.0
8228,8228,mr. A.G.A. ridder van Rappard,NaN,NaN,enige kandidaat,Amersfoort,18/04/1918,tussentijds/enkelvoudig,14218,-,NaN,-,NaN,-,0.0,999.0


# Important

Now, we must merge this dataframe twice, that is to say, with two dataframes:
    
- 1. The data needs to be merged with the non-politicians (non-politicians left): so as to get the margins for non-politicians

These observations are then ready to be concatenated with the politicians (step 3). 

- 2. The data needs to be filtered to include only close elections for politicians, and then string matched to the politicians in `politician_wealth.csv` to find their wealth. 

- 3. Concatenate the two data frames. 

In [10]:
# First, we implement 1.
nonpoliticians_margin = nonpoliticians.merge(df_margins, left_on = ['Naam', 'District', 'Verkiezingdatum'],
                    right_on = ['Naam', 'District', 'Verkiezingdatum'], how = "left")

nonpoliticians_margin.head(2)

,Naam,District,Verkiezingdatum,InTKEK,Gevonden,Gebdatum,Sterfdatum,Sterfplaats,Vermogen,Link1,...,Procentueel,Type,Omvang electoraat,Aantal stembriefjes,Aantal stemmen geldig,Aantal stemmen blanco,Aantal zetels,Kiesdrempel,Opkomst,margin
0,mr. J. van Riemsdijk,Almelo,02/01/1849,nee,ja,NaN,01/11/1879,Almelo,29618.27,https://www.genealogieonline.nl/en/genealogie-...,...,44.38%,herstemming,547.0,481,480.0,1,1.0,240,480.0,-0.112500
1,mr. H. van Sonsbeeck,Enschede,30/11/1848,nee,nee,NaN,29/11/1865,Heino,NaN,NaN,...,44.33%,algemeen,532.0,487,485.0,1,1.0,243,476.0,-0.096639


# Implementation of 2.

Implementation of 2 is a little bit more difficult. All close winners are:

- Taking part in the same election as all close losers
- At the same filtered position as the number of seats (-1 in Python index) OR
- Having a close but positive margin



In [11]:
# Taking part in the same election as all close losers:
closeels = nonpoliticians.groupby(['District', 'Verkiezingdatum']).groups
politicians_margin = df_margins[df_margins.apply(lambda row: (row["District"], row["Verkiezingdatum"]) in closeels, axis=1)]

In [12]:
# At the same filtered position as the number of seats (-1 in Python index) or having a close but positive margin
politicians_margin = politicians_margin[politicians_margin['margin'].between(0, 0.20)]

In [13]:
politicians_margin.head(2)

,Unnamed: 0,Naam,Aanbevolen door,Aantal stemmen,Procentueel,District,Verkiezingdatum,Type,Omvang electoraat,Aantal stembriefjes,Aantal stemmen geldig,Aantal stemmen blanco,Aantal zetels,Kiesdrempel,Opkomst,margin
13,13,mr. R. de Sitter,NaN,257.0,35.64%,Winschoten,30/11/1848,algemeen,1183,722,721.0,1,1.0,361,696.0,0.071839
21,21,dr. R. Westerhoff,NaN,225.0,32.89%,Appingedam,27/12/1848,tussentijds,1586,687,684.0,3,1.0,342,674.0,0.136499


## Implementation 2 finalization

Finally, we have to match their names using the same function as in `data_to_analysis_matched.ipynb`, through which we get the b1_nummers, after which we can merge the politicians with their wealth in `politicians_wealth.csv`. 

We also add match_score to each iteration, so it becomes easier to filter on this variable later. 


In [14]:
# df (supposed to be the dataframe politicians_margin) has to contain Naam = name of the politician
def find_polid(df):
    
    df.reset_index(drop = True, inplace = True)
    ## Change some stupid things in Naam
    df['Naam'] = df['Naam'].apply(lambda x: " ".join(re.sub("baron|ridder|jhr.|mr.|jhr.mr.|dr.", "", 
                                     x).split()))
    
    ## Load the data
    polidlist = pd.read_excel("../Data/politician_data/tk_1815tot1950uu.xlsx", dtype={'b1-nummer':str})
    polidlist['voorlachternaam'] = polidlist['voorletters'] + ' ' + polidlist['achternaam']
    
    matched_b1_nummer = []
    matched_score = []

    for i in tqdm(range(len(df))):
        
        nameofpoliticianindf = df.iloc[i]['Naam']
        listofpotmatches = polidlist['voorlachternaam']
        
        # Match attempt I
        nameofpoliticianindf_2 = " ".join(re.sub("Van De |Van Der |van de |van der |van den |van |Van der |Van |de ", "", 
                                      nameofpoliticianindf).split())
        
        match_prelim = match.extractOne(nameofpoliticianindf_2, listofpotmatches, match_type='jaro_winkler')
        
        if match_prelim[1] > 0.85:
            match_defin = match_prelim[0]
            #Find the b1-nummer and append that to the b1_matched column
            matched_b1_nummer.append(polidlist[polidlist['voorlachternaam'] == match_defin]['b1-nummer'].iloc[0])
            matched_score.append(match_prelim[1])
            continue
        
        # Match attempt II
        match_prelim = match.extractOne(nameofpoliticianindf, listofpotmatches, match_type="jaro_winkler")        
        
        if match_prelim[1] > 0.82:
            match_defin = match_prelim[0]
            matched_b1_nummer.append(polidlist[polidlist['voorlachternaam'] == match_defin]['b1-nummer'].iloc[0])
            matched_score.append(match_prelim[1])
            continue
            
        # Match attempt III - Update list of matches
        lowerlimit = pd.Timestamp(df.iloc[i]['Verkiezingdatum'])
        listofpotmatches = polidlist[polidlist['einde periode'].apply(lambda x: pd.Timestamp(x)) > lowerlimit]
        listofpotmatches = listofpotmatches[listofpotmatches['begin periode'].apply(lambda x: pd.Timestamp(x)) < (lowerlimit + pd.DateOffset(days=365))]
        listofpotmatches = listofpotmatches['voorlachternaam']
        
        match_prelim = match.extractOne(nameofpoliticianindf, listofpotmatches, match_type="jaro_winkler")
        #print(i, df.iloc[i]['Naam'], match_prelim)
        if match_prelim[1] > 0.67:
            match_defin = match_prelim[0]
            matched_b1_nummer.append(polidlist[polidlist['voorlachternaam'] == match_defin]['b1-nummer'].iloc[0])
            matched_score.append(match_prelim[1])
            continue
            
        else:
            match_defin = "No match found"
            matched_b1_nummer.append(99999)
            matched_score.append(0
                                )
# Finally (outside the for-loop), append the b1_matched column to the dataframe and then merge it (left_join)
    df['matched b1-nummer'] = pd.Series(matched_b1_nummer)
    df['matched_score'] = pd.Series(matched_score)
    
    df = df.merge(polidlist, how="left", left_on = ["matched b1-nummer"], right_on="b1-nummer")

    return df

In [15]:
polb1no = find_polid(politicians_margin)

polb1no.tail(3)

100%|██████████| 597/597 [00:27<00:00, 21.62it/s]


,Unnamed: 0,Naam,Aanbevolen door,Aantal stemmen,Procentueel,District,Verkiezingdatum,Type,Omvang electoraat,Aantal stembriefjes,...,achternaam,voorletters,voorna(a)m(en),roepnaam,prepositie,geslacht,partij(en)/fractie(s),begin periode,einde periode,voorlachternaam
594,8220,A.B. de Zeeuw,NaN,3628.0,39.20%,Rotterdam II,14/12/1917,tussentijds,17453,9375,...,Zeeuw,A.B.,Arie Bastiaan,Arie,de,m,SDAP,1918-01-15,1919-10-31,A.B. Zeeuw
595,8224,A.B. de Zeeuw,NaN,4844.0,50.92%,Rotterdam II,21/12/1917,herstemming,17453,9591,...,Zeeuw,A.B.,Arie Bastiaan,Arie,de,m,SDAP,1918-01-15,1919-10-31,A.B. Zeeuw
596,8234,D.J. de Geer,NaN,3667.0,53.31%,Schiedam,22/10/1907,herstemming,8083,6916,...,Geer,D.J.,Dirk Jan,Dirk,de,m,CHP;CHU,1907-11-04,1939-08-09,D.J. Geer


## Add Wealth

Now, we merge this dataframe with the `wealth_politicians.csv` document.

In [16]:
pd.set_option('display.max_rows', None)

wealth = pd.read_csv("../Data/politician_data/wealth_politicians.csv", dtype = {'indexnummer':str})

politicians_final = polb1no.merge(wealth, left_on=['matched b1-nummer'], right_on=['indexnummer'], how = 'left')

politicians_final.columns

Index(['Unnamed: 0', 'Naam', 'Aanbevolen door', 'Aantal stemmen',
       'Procentueel', 'District', 'Verkiezingdatum', 'Type',
       'Omvang electoraat', 'Aantal stembriefjes', 'Aantal stemmen geldig',
       'Aantal stemmen blanco', 'Aantal zetels', 'Kiesdrempel', 'Opkomst',
       'margin', 'matched b1-nummer', 'matched_score', 'b1-nummer',
       'achternaam', 'voorletters', 'voorna(a)m(en)', 'roepnaam', 'prepositie',
       'geslacht', 'partij(en)/fractie(s)', 'begin periode', 'einde periode',
       'voorlachternaam', 'indexnummer', 're', 'dugobo', 'fogobo', 'duprbo',
       'foprbo', 'dush', 'fosh', 'cash', 'misc', 'debt', 'ta11', 'tl11',
       'nw11', 'ta0', 'tl0', 'nw0', 'w_deflated'],
      dtype='object')

# Finally

Concatenate `politicians_final` and `nonpoliticians_margin` to obtain a dataframe with close winners and close losers, and the margin (and the b1_nummer for politicians). 

In [17]:
a = politicians_final[['b1-nummer','Naam', 'District', 'Verkiezingdatum', 'nw0', 'margin']].rename(columns={'nw0':'Vermogen'})
b = nonpoliticians_margin[['Naam', 'District', 'Verkiezingdatum','Sterfdatum','Vermogen','margin']]

## Addition

We have to deflate the numbers. We have to do this in three steps:

- Find the DoD For Politicians, add them to politicians_final
- Find the DoD for Nonpoliticians, add them to nonpoliticians_margin (already done in previous chunk)

- Apply the function to get the deflated wealth (second chunk below)

- Then, we export it again to csv

In [18]:
deathdates_pol = pd.read_excel("../Data/politician_data/tk_1815tot1950uu.xlsx", sheet_name = 1, dtype={'b1-nummer': str})
deathdates_pol = deathdates_pol[deathdates_pol['rubriek'] == 3020][['b1-nummer','datum']]

In [19]:
a = pd.merge(a, deathdates_pol, how="left", left_on = "b1-nummer", right_on = "b1-nummer").rename(columns={'datum':'Sterfdatum'})

In [20]:
# Now, we deflate Vermogen: 
def tryextract(x):
    
    try: 
        out = pd.to_numeric(x[2])
    except:
        out = 999
    return(out)
   
# We deflate the wealth numbers
def deflate(df): 

    # import the deflator dataset
    deflator = read_ods("../Data/politician_data/memories_invoer.ods", sheet = 3).iloc[:,[0,2,3]]
    
    # create a year variable from Sterfdatum
    df['Year'] = df['Sterfdatum'].str.split('-|/').apply(lambda x: tryextract(x))
    
    wealth_def = []
    
    for i in range(len(df)):
        try:
            multiply = deflator[deflator['Year'] == pd.to_numeric(df['Year'][i])].iloc[0].iloc[2]
            wealth_def.append(pd.to_numeric(df['Vermogen'][i]) * multiply)
            
        except: 
            wealth_def.append(None)
            
    df['Vermogen_deflated'] = pd.Series(wealth_def)
    #delete year again
    del df['Year'] 
    
    return(df)

In [21]:
# This file can be exported to csv
final = pd.concat([a, b]).reset_index(drop = True)
final = deflate(final)
final.shape

(1271, 8)

In [22]:
final.to_csv("../Data/analysis/unmatched_sample_analysis.csv")